In [9]:
import json
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential
import os, requests
from dotenv import load_dotenv


In [3]:
body = {
    "values": [
        {   "recordId": "record1",
            "data": { 
                "key":"aHR0cHM6Ly9mcnN0b3JhZ2UyMDIzLmJsb2IuY29yZS53aW5kb3dzLm5ldC9zY2llbnN1cy9SeCUyMDYucG5n0",
                "metadata_storage_path":"aHR0cHM6Ly9mcnN0b3JhZ2UyMDIzLmJsb2IuY29yZS53aW5kb3dzLm5ldC9zY2llbnN1cy9SeCUyMDYucG5n0",
                "metadata_storage_path_decoded":"https://frstorage2023.blob.core.windows.net/sciensus/Rx%207.png",
                "formUrl": "https://frstorage2023.blob.core.windows.net/sciensus/Rx%207.png",
                "formSasToken": "?sv=2021-10-04&st=2023-02-10T10%3A05%3A29Z&se=2025-10-31T10%3A05%3A00Z&sr=b&sp=r&sig=PxoN9tDIrJVF7NM4d8tXEE773um06OnB1jgMtyQ%2F6%2FM%3D",
                "metadata_storage_sas_token":"?sv=2021-10-04&st=2023-02-10T10%3A05%3A29Z&se=2025-10-31T10%3A05%3A00Z&sr=b&sp=r&sig=PxoN9tDIrJVF7NM4d8tXEE773um06OnB1jgMtyQ%2F6%2FM%3D"
                }
        }
    ]
}
json_data = json.dumps(body)
values = json.loads(json_data)['values'] 

In [29]:
str(body['values'][0]['data'])



"{'metadata_storage_path': 'https://frstorage2023.blob.core.windows.net/sciensus/Rx%207.png', 'metadata_storage_path_decoded': 'https://frstorage2023.blob.core.windows.net/sciensus/Rx%207.png', 'formUrl': 'https://frstorage2023.blob.core.windows.net/sciensus/Rx%207.png', 'formSasToken': '?sv=2021-10-04&st=2023-02-09T09%3A49%3A18Z&se=2023-02-10T09%3A49%3A18Z&sr=b&sp=r&sig=NBABXHCPGq8Z9lNrGuTEEKFoDEUGQD72IUbTTtHNCNw%3D', 'metadata_storage_sas_tokem': '?sv=2021-10-04&st=2023-02-09T09%3A49%3A18Z&se=2023-02-10T09%3A49%3A18Z&sr=b&sp=r&sig=NBABXHCPGq8Z9lNrGuTEEKFoDEUGQD72IUbTTtHNCNw%3D'}"

In [34]:
body = {
    "values": [
        {   "recordId": "record1",
            "data": { 
                "metadata_storage_path":"https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png",
                "metadata_storage_path_decoded":"https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png",
                "formUrl": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png",
                "formSasToken": "",
                "metadata_storage_sas_token":""
                }
        }
    ]
}
json_data = json.dumps(body)
values = json.loads(json_data)['values'] 

## Adding FR metadata 

In [35]:
load_dotenv('..')
cog_endpoint = os.getenv('COGNITIVE_SERVICE_FORM_RECOGNIZER_ENDPOINT')
cog_key = os.getenv('COGNITIVE_SERVICE_FORM_RECOGNIZER_KEY')
model= "prebuilt-layout"
form_recognizer_url = f"{cog_endpoint}formrecognizer/documentModels/{model}:analyze?api-version=2022-08-31"


## Here comes the transform_value part



In [36]:
# extracting one value metadata
value=values[0]
data = value["data"]
form_url = data["formUrl"]  + data["formSasToken"]   
recordId = value['recordId']


# Making a placeholder for the service bus message
output=[]
output = {
            "key": "",
            "file_path": form_url,
            "model": model,
            "Operation-location":""
        } 

# Request to Azure Form Recognizer Model
headers = {"Content-Type": "application/json", "Ocp-Apim-Subscription-Key": cog_key}
body = {'urlSource': output['file_path']}
r = requests.post(form_recognizer_url, headers=headers, json=body)

print(r.status_code)
# Adding last bit to the service bus message
if r.status_code<300:
    output['Operation-location'] = r.headers['Operation-Location']
else: output['Operation-location']=f"code: {r.status_code}"
json.dumps(output)


202


'{"key": "", "file_path": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png", "model": "prebuilt-layout", "Operation-location": "https://formr-pdf.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/c281178f-87f4-4ffd-89ac-11fe548d4ad7?api-version=2022-08-31"}'

In [37]:
requests.post(form_recognizer_url, headers=headers, json=body)

<Response [202]>

In [17]:
tables = []
for i in range (2):
    tables.append({"page_number": 'p'+str(i),
                    "raw_count" : '122',
                    "column_count": '23',
                    "headers": [2,3,4],
                    "cells": ['a','b','c'],
                    "raw_text":'abc'}) 
# tables is a list of dictionaries


[{'page_number': 'p0',
  'raw_count': '122',
  'column_count': '23',
  'headers': [2, 3, 4],
  'cells': ['a', 'b', 'c'],
  'raw_text': 'abc'},
 {'page_number': 'p1',
  'raw_count': '122',
  'column_count': '23',
  'headers': [2, 3, 4],
  'cells': ['a', 'b', 'c'],
  'raw_text': 'abc'}]

In [21]:
results = {  
        "value": [  
            {  
                "@search.action": "merge",  
                "metadata_storage_path": 'path_from_data'
            }  
        ]  
    }

results['value'][0]['tables']=tables
results

{'value': [{'@search.action': 'merge',
   'metadata_storage_path': 'path_from_data',
   'tables': [{'page_number': 'p0',
     'raw_count': '122',
     'column_count': '23',
     'headers': [2, 3, 4],
     'cells': ['a', 'b', 'c'],
     'raw_text': 'abc'},
    {'page_number': 'p1',
     'raw_count': '122',
     'column_count': '23',
     'headers': [2, 3, 4],
     'cells': ['a', 'b', 'c'],
     'raw_text': 'abc'}]}]}